In [39]:
from sage.all import *
from sage.matroids.advanced import *

In [5]:
M = matroids.Wheel(4)
print(M)
M.is_connected()

Wheel(4): Regular matroid of rank 4 on 8 elements with 45 bases


True

In [7]:
U36 = matroids.Uniform(3, 6)
U36.equals(U36.dual())

True

In [8]:
F7 = matroids.catalog.Fano()
len(F7.nonspanning_circuits())

7

In [14]:
matroids.catalog.Fano()

Fano: Binary matroid of rank 3 on 7 elements, type (3, 0)

In [40]:
# Fano matroid
A = Matrix(GF(2),[[1, 0, 0, 0, 1, 1, 1],[0, 1, 0, 1, 0, 1, 1],[0, 0, 1, 1, 1, 0, 1]])
try:
    M = Matroid(A)
    print(M)
    print(M.is_isomorphic(matroids.catalog.Fano()))
except NotImplementedError as e:
        print(e)

Binary matroid of rank 3 on 7 elements, type (3, 0)
True


In [72]:
# (complete graphic) matroid from a matrix
A = Matrix(GF(2), [[1, 0, 0, 1, 1, 0],[0, 1, 0, 1, 0, 1],[0, 0, 1, 0, 1, 1]])
M = Matroid(matrix=A)
M.is_isomorphic(matroids.CompleteGraphic(4))

True

In [46]:
# matroid associated to Peterson graph
G = graphs.PetersenGraph()
M_G = Matroid(G)
M_G.rank(), MG.circuits()

(9, SetSystem of 57 sets over 15 elements)

In [63]:
# matroids constructed from sets
M1 = Matroid(groundset='abcd', bases=['ab', 'ac', 'ad','bc', 'bd', 'cd']); print(M1)
M2 = Matroid(bases=['ab', 'ac', 'ad', 'bc', 'bd', 'cd']); print(M2)
M3 = Matroid(['ab', 'ac', 'ad', 'bc', 'bd', 'cd']); print(M3)
M4 = Matroid('abcd', ['ab', 'ac', 'ad', 'bc', 'bd', 'cd']); print(M4)
M5 = Matroid('abcd', bases=[['a', 'b'], ['a', 'c'],['a', 'd'], ['b', 'c'],['b', 'd'], ['c', 'd']]); print(M5)
print(M1 == M2)
print(M1 == M3)
print(M1 == M4)
print(M1 == M5)

Matroid of rank 2 on 4 elements with 6 bases
Matroid of rank 2 on 4 elements with 6 bases
Matroid of rank 2 on 4 elements with 6 bases
Matroid of rank 2 on 4 elements with 6 bases
Matroid of rank 2 on 4 elements with 6 bases
True
True
True
True


In [69]:
M = Matroid(flats={0: [''], 1: ['a', 'b'], 2: ['ab']})
print(M.is_isomorphic(matroids.Uniform(2, 2)) and M.is_valid())
print(type(M))

True
<class 'sage.matroids.flats_matroid.FlatsMatroid'>


In [70]:
G = Graph([(0, 1, 'a'), (0, 2, 'b'), (1, 2, 'c')])     
M = Matroid(G)
sorted(M.groundset())

['a', 'b', 'c']

In [91]:
#matroid example from "The Geometry of Matroids"
M_E = Matroid(groundset='abcde', bases=['abc', 'abd', 'abe','acd', 'ace']); print(M)
print(M_E.is_graphic())
G = Graph([(0,1,'a'), (1,2,'b'), (1,3,'c'), (2,3,'d'), (2,3,'e')]); print(G)
M_G = Matroid(G); print(M_G)
M_E.is_isomorphic(M_G)

Matroid of rank 3 on 5 elements with 5 bases
True
Graph on 4 vertices
Graphic matroid of rank 3 on 4 elements


False